In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 13.06 GB
Used RAM: 2.09 GB
Percentage Usage Of RAM: 15.6%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 17.02 GB
Used Disk: 7.96 GB
Percentage Usage Of Disk: 31.9%


In [3]:
# import requirements
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from wrangle import prepare
from lm_classifier import Classification

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# get the data
titanic = pd.read_csv("titanic.csv")

In [5]:
# prepare the data for machine learning
titanic = prepare(df=titanic)

Data Wrangling:
> Removing Unnecessary Columns
> Getting Titles From Name
> Transforming Ticket
> Replacing Missing Values
> Transforming Cabin
> Shuffling The Data
0.02 Seconds


In [6]:
# get the testing data
y = titanic[["Survived"]]
X = titanic.drop(columns="Survived")
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [7]:
# build the model
print("\n---- Titanic Classification Analysis ----\n")
model = Classification(
    name="Lasso Without Feature Engineering", 
    path=None,
    rename=False, 
    time=False, 
    text=False,
    binary=True, 
    imputation=True, 
    variance=True,
    scale=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(titanic)
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"Accuracy: {model.accuracy}")
    print(f"F1: {model.f1}")
    print(f"In Control: {model.in_control}")
    print("Confusion Matrix:")
    pd.set_option("display.width", 1000)
    print(model.confusion / model.confusion.sum().sum())


---- Titanic Classification Analysis ----

Visualizing The Data:
> Plotting Correlations
> Plotting Age
> Plotting Fare
> Plotting Sex
> Plotting Ticket
> Plotting Cabin
> Plotting Embarked
> Plotting Title
> Plotting Survived
> Plotting Pclass
> Plotting SibSp
> Plotting Parch
> Sex vs. Ticket
> Sex vs. Cabin
> Sex vs. Embarked
> Sex vs. Title
> Sex vs. Survived
> Sex vs. Pclass
> Sex vs. SibSp
> Sex vs. Parch
> Ticket vs. Cabin
> Ticket vs. Embarked
> Ticket vs. Title
> Ticket vs. Survived
> Ticket vs. Pclass
> Ticket vs. SibSp
> Ticket vs. Parch
> Cabin vs. Embarked
> Cabin vs. Title
> Cabin vs. Survived
> Cabin vs. Pclass
> Cabin vs. SibSp
> Cabin vs. Parch
> Embarked vs. Title
> Embarked vs. Survived
> Embarked vs. Pclass
> Embarked vs. SibSp
> Embarked vs. Parch
> Title vs. Survived
> Title vs. Pclass
> Title vs. SibSp
> Title vs. Parch
> Survived vs. Pclass
> Survived vs. SibSp
> Survived vs. Parch
> Pclass vs. SibSp
> Pclass vs. Parch
> SibSp vs. Parch
> Age vs. Sex
> Age vs. 

In [14]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10].tolist()):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10].tolist()):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. Title_Master.
2. SibSp_2
3. SibSp_0
4. Age
5. SibSp_1
6. Ticket_W
7. Sex_male
8. Parch_4
9. Title_Sir.
10. Fare
 
Feature Drift:
None


In [15]:
# score the model
accuracy = accuracy_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)
f1 = f1_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    average="macro",
)

print(f"Accuracy: {accuracy}")
print(f"F1: {f1}")

Accuracy: 0.8651685393258427
F1: 0.8634445723053319


In [10]:
# show the confusion matrix
ytest = testy.iloc[:,0].to_numpy()
labels = np.unique(np.concatenate((predictions, ytest)))
confusion = confusion_matrix(
    y_true=ytest,   # rows
    y_pred=predictions,  # columns
    labels=labels,
)
confusion = pd.DataFrame(
    confusion, 
    columns=labels, 
    index=labels,
)
print("Confusion Matrix:")
pd.set_option("display.width", 1000)
print(confusion / confusion.sum().sum())

Confusion Matrix:
          0         1
0  0.488764  0.056180
1  0.078652  0.376404


In [11]:
# save the machine learning pipeline
model.dump()

In [12]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Transforming Categorical Features
> Filling In Missing Values
> Removing Constant Features
> Scaling Features
> Training Lasso
3.36 Seconds
Model Indicators:
> Extracting Important Features
1.09 Seconds
